# Data Science Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [1]:
# Use this cell to write your code for Task 1
import pandas as pd

data = pd.read_csv("house_sales.csv")
data['city'].value_counts()
missing_city = 73
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [2]:
# Use this cell to write your code for Task 2

house_sales = pd.read_csv("house_sales.csv")

house_sales['house_id'] = house_sales['house_id'].astype('category') 

house_sales['area'] = house_sales['area'].str.strip(' sq.m.')
house_sales['area'] = house_sales['area'].astype('float')

house_sales['city'] = house_sales['city'].str.replace('--', "Unknown")
house_sales['city'] = house_sales['city'].astype('category') 

house_sales["sale_date"] = house_sales["sale_date"].fillna("2023-01-01")
house_sales['sale_date'] = pd.to_datetime(house_sales['sale_date'])

house_sales['months_listed'] = house_sales['months_listed'].fillna(house_sales["months_listed"].mean()).round(1)

house_sales["bedrooms"] = house_sales["bedrooms"].fillna(round(house_sales["bedrooms"].mean())).astype(int)

house_sales["house_type"] = house_sales["house_type"].astype('category')
house_type_mapping = {
    'Terr.': 'Terraced',
    'Semi': 'Semi-detached',
    'Det.': 'Detached'
}
house_sales["house_type"] = house_sales["house_type"].replace(house_type_mapping)
house_sales["house_type"].value_counts(dropna=False)

mean_area = house_sales["area"].mean()
house_sales["area"].fillna(round(mean_area, 1), inplace=True)

clean_data = house_sales
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   house_id       1500 non-null   category      
 1   city           1500 non-null   category      
 2   sale_price     1500 non-null   int64         
 3   sale_date      1500 non-null   datetime64[ns]
 4   months_listed  1500 non-null   float64       
 5   bedrooms       1500 non-null   int64         
 6   house_type     1500 non-null   category      
 7   area           1500 non-null   float64       
dtypes: category(3), datetime64[ns](1), float64(2), int64(2)
memory usage: 108.9 KB


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [3]:
# Use this cell to write your code for Task 3

import pandas as pd

house_sales = pd.read_csv("house_sales.csv")
price_by_rooms = house_sales.groupby("bedrooms")["sale_price"].agg(["mean", "var"]).reset_index()
price_by_rooms.columns = ["bedrooms", "avg_price", "var_price"]
price_by_rooms = price_by_rooms.round(1)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [4]:
# Use this cell to write your code for Task 4
import pandas as pd
data = pd.read_csv("train.csv")
train_df=data.copy()
validation = pd.read_csv("validation.csv")
validation_df = validation.copy()

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer


features = ["city", "months_listed", "bedrooms", "house_type", "area"]
target = "sale_price"


X_train = train_df[features]
y_train = train_df[target]
X_val = validation_df[features]


categorical_features = ["city", "house_type"]
numeric_features = ["months_listed", "bedrooms", "area"]

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)


model = make_pipeline(preprocessor, LinearRegression())

model.fit(X_train, y_train)

validation_df["price"] = model.predict(X_val)

base_result = validation_df[["house_id", "price"]]
base_result.head()


,house_id,price
0,1331375,121527.827316
1,1630115,304386.625267
2,1645745,384760.100656
3,1336775,123976.268985
4,1888274,271186.199353


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [5]:
# Use this cell to write your code for Task 5
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd

train_data = pd.read_csv("train.csv")


features = ["bedrooms", "months_listed", "area"]
target = "sale_price"

X_train = train_data[features]
y_train = train_data[target]

model = make_pipeline(StandardScaler(), LinearRegression())
model.fit(X_train, y_train)

validation_data = pd.read_csv("validation.csv")

X_validation = validation_data[features]
validation_data["price"] = model.predict(X_validation)

compare_result = validation_data[["house_id", "price"]]
compare_result = compare_result.round(1)

compare_result.head()


,house_id,price
0,1331375,118876.1
1,1630115,257484.2
2,1645745,383988.5
3,1336775,117832.4
4,1888274,256378.7
